# Model Conversion Notebook: nnU-Net to Core ML

This notebook guides you through setting up the environment, downloading the necessary nnU-Net model weights, and converting them to the Core ML format (`.mlmodel`) required for the EchoAnalyzer iOS app.

**Prerequisites:**
- A Python 3 environment.
- Internet access.
- The `plans.json` file should be placed in the correct relative path for `nnunet_model_conversion_final.py` to find it. The script expects it at: `EchoAnalyzer/EchoAnalyzer/nnU-Net_CAMUS_EchoGAINS/nnUNetTrainer__nnUNetPlans__2d/plans.json`. Please ensure this path exists relative to where you run this notebook, or adjust the path in the conversion script cell.

In [ ]:
!pip install torch torchvision torchaudio
!pip install coremltools onnx onnx-simplifier tqdm gdown requests

## Fetch Utility Scripts

The `download_model.py` script will be fetched from the user's repository.
The `nnunet_model_conversion_final.py` script (prepared in a previous step) should be available in the workspace and will be saved alongside this notebook.

In [ ]:
import os
import requests

download_script_url = "https://raw.githubusercontent.com/devilog4n/Echo_Weights/main/download_model.py"
download_script_name = "download_model.py"

if not os.path.exists(download_script_name):
    print(f"Downloading {download_script_name} from {download_script_url}...")
    try:
        response = requests.get(download_script_url)
        response.raise_for_status()
        with open(download_script_name, 'wb') as f:
            f.write(response.content)
        print(f"{download_script_name} downloaded successfully.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading {download_script_name}: {e}")
        print(f"Please ensure {download_script_name} is manually placed in the same directory as this notebook if download fails.")
else:
    print(f"{download_script_name} already exists locally.")

# Ensure nnunet_model_conversion_final.py is also present (worker should place it here)
conversion_script_name = "nnunet_model_conversion_final.py"
if not os.path.exists(conversion_script_name):
    print(f"ERROR: {conversion_script_name} not found. This script should have been provided by the AI worker.")
else:
    print(f"{conversion_script_name} is present.")

print("\nRequired scripts should now be in the current directory.")

## Prepare `plans.json` Path

The conversion script `nnunet_model_conversion_final.py` needs the `plans.json` file.
Its expected path relative to this notebook's directory is:
`EchoAnalyzer/EchoAnalyzer/nnU-Net_CAMUS_EchoGAINS/nnUNetTrainer__nnUNetPlans__2d/plans.json`

1.  **Manually create this directory structure** if it doesn't exist from where you are running this notebook.
2.  **Place your `plans.json` file** into that exact location.

In [ ]:
import os
plans_json_expected_dir = "EchoAnalyzer/EchoAnalyzer/nnU-Net_CAMUS_EchoGAINS/nnUNetTrainer__nnUNetPlans__2d/"
if not os.path.exists(plans_json_expected_dir):
    os.makedirs(plans_json_expected_dir)
    print(f"Created directory structure: {os.path.abspath(plans_json_expected_dir)}")
else:
    print(f"Directory structure already exists: {os.path.abspath(plans_json_expected_dir)}")

print(f"Please ensure 'plans.json' is placed inside: {os.path.abspath(plans_json_expected_dir)}plans.json")

## Download nnU-Net Model Weights

The `download_model.py` script will attempt to use local model files from a `./models/` directory first, or download them if not found.
We are interested in the nnU-Net segmentation model, typically `checkpoint_best.pth` or `checkpoint_final.pth`.
The script might require an argument like 'camus', 'best', or 'final' to target the correct model.

In [ ]:
import os
# Ensure the models directory exists
models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)
print(f"Place pre-downloaded .pth model files in: '{os.path.abspath(models_dir)}' for local priority.")

print("\nAttempting to run download_model.py...")
# Try common arguments for the nnU-Net segmentation model.
# User might need to adapt based on download_model.py's exact CLI.
# Option 1: if script automatically looks for local files or has a 'camus' default for segmentation
# !python download_model.py
# Option 2: Explicitly try to download 'best' or 'final' checkpoint if that's how the script works
!python download_model.py best || python download_model.py final || python download_model.py camus

print("\nPlease check if 'checkpoint_best.pth' or 'checkpoint_final.pth' is now in the './models/' directory.")
print("If not, please run download_model.py manually from your terminal with the correct arguments, or place the .pth file there.")

## Convert to Core ML

This cell runs `nnunet_model_conversion_final.py`.
It assumes:
- The PyTorch model (e.g., `checkpoint_best.pth`) is in `./models/`.
- `plans.json` is at `EchoAnalyzer/EchoAnalyzer/nnU-Net_CAMUS_EchoGAINS/nnUNetTrainer__nnUNetPlans__2d/plans.json`.
- It will generate `EchoSegmenter.mlmodel` in the current directory.

In [ ]:
print("Attempting to convert the model to Core ML...")
!python nnunet_model_conversion_final.py

print("\nConversion attempt finished.")
print("Check for 'EchoSegmenter.mlmodel' in the current directory.")

## Next Steps

1.  If `EchoSegmenter.mlmodel` was generated successfully, add it to your `EchoAnalyzer` Xcode project.
2.  Ensure it's included in the target's "Copy Bundle Resources" build phase.
3.  Build and run the iOS app.

If you encountered Python errors, ensure all dependencies (`torch`, `coremltools`, `onnx`, `onnx-simplifier`, `tqdm`, `gdown`, `requests`) are correctly installed in your notebook's Python environment.
The `nnunet_model_conversion_final.py` script (which should be in the same directory as this notebook) contains the conversion logic using parameters from `plans.json`. The `download_model.py` (also in this directory) is used to fetch the weights.